In [1]:
%cd project/

/home/rapids/notebooks/project


In [10]:
# delete pip package
%pip uninstall -y scikit-mobility


Found existing installation: scikit-mobility 1.2.2
Uninstalling scikit-mobility-1.2.2:
  Successfully uninstalled scikit-mobility-1.2.2
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# conda install requirments
# %conda install -y -c conda-forge scikit-mobility
%conda list

In [ ]:
%env SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True
%pip install -r "requirements.txt"

In [2]:
import skmob
from skmob.utils import utils, constants
from skmob.tessellation import tilers
from skmob.utils.plot import plot_gdf

import numpy as np
import pandas as pd
import geopandas as gpd
from skmob.models.gravity import Gravity


from sklearn.model_selection import train_test_split
from random import sample
import pickle
from scipy.spatial import distance
import sys

import time

from matplotlib import pyplot as plt
plt.rcParams["figure.figsize"] = (12, 9) # (w, h)

UTILS

In [3]:
def toFdf(adj, tess, d):
    l = []
    for i in range(0,len(adj)):
        for j in range(0,len(adj)):
            if(adj[i,j]!=0):
                l.append([d[i],d[j], adj[i,j]])
    data = pd.DataFrame(l)
    data[0] = data[0].astype(str)
    return(data)

In [4]:
def toAdj(fdf, tess):
    names = list(tess.tile_ID.values)
    adj = pd.DataFrame(columns=names, index = names)
    for i, row in enumerate(fdf.itertuples(), 1):
         adj.at[str(row.origin), str(row.destination)] = row.flow
    return adj

TESSELLATION

In [5]:
shape_file_land = gpd.read_file("./barriosCABA.geojson")

meters = 2612.61
tessellation = tilers.tiler.get("squared", meters=meters, base_shape=shape_file_land)
tessellation.crs="EPSG:4326"
plot_gdf(tessellation, style_func_args={'fillColor':'gray', 'color':'black', 'opacity': 0.2}, zoom = 10) 


/opt/conda/lib/python3.11/site-packages/skmob/tessellation/tilers.py:117: ShapelyDeprecationWarning: The 'cascaded_union()' function is deprecated. Use 'unary_union()' instead.
  base_shape = gpd.GeoSeries(cascaded_union(polygons), crs=base_shape.crs)


DATA

In [6]:
with open("./generations/dia/v_train.txt", "rb") as fp:   # Unpickling
    v_train = pickle.load(fp)
# v_train = v_train.tolist()
print(len(v_train))


with open("./generations/dia/v_test.txt", "rb") as fp:   # Unpickling
    v_test = pickle.load(fp)
# v_test = v_test.tolist()
print(len(v_test))

# with open("./generations/dia/v_train_2.txt", "rb") as fp:   # Unpickling
#     v_train2 = pickle.load(fp)
# # v_test = v_test.tolist()
# print(len(v_train2))

1380
346


GRAVITY

In [8]:
a = list(enumerate(list(tessellation.tile_ID.values)))
d = dict(a)
tiles = np.asarray(list(d.values())).astype(int)
zeros = np.zeros(tiles.size)

np.random.seed(0)
a = 0
l = []

start = time.time()

for n in sample(v_train, len(v_test)):

    gravity_singly_fitted = Gravity(gravity_type='singly constrained')

    lookup = dict(zip(tiles.astype(str), zeros))

    lookup_pick = dict(zip(tiles.astype(str), zeros))

    flow = toFdf(np.asarray(n), tessellation, d)
    flow = flow[flow[0] != flow[1]]

    z = flow.groupby([0])[2].sum()
    for el in z.keys():
        lookup[el] += z[el]

    flow_pick = toFdf(np.asarray(n), tessellation, d)
    z = flow_pick.groupby([0])[2].sum()
    for el in z.keys():
        lookup_pick[el] += z[el]

    # print(tessellation)
    tessellation['tile_ID'] = tessellation['tile_ID'].astype('str')

    
    # fdf['origin'] = fdf['origin'].astype('int64')
    # fdf['destination'] = fdf['destination'].astype('int64')
    # fdf.tessellation[constants.TILE_ID] = fdf.tessellation[constants.TILE_ID].astype('int64')
    
    # print(fdf)
    # tessellation['tot_outflow'] = tessellation['tile_ID'].map(lambda x: lookup[str(x)])
    # tessellation['relevance'] = tessellation['tile_ID'].map(lambda x: lookup_pick[str(x)])
    tessellation['tot_outflow'] = tessellation['tile_ID'].map(lookup)
    tessellation['relevance'] = tessellation['tile_ID'].map(lookup_pick)
    tessellation['relevance'] = tessellation['relevance'] / (tessellation['relevance'].max())
    tessellation['relevance'] = tessellation['relevance'] + 0.00001

    fdf = skmob.FlowDataFrame(data = flow,tessellation=tessellation, tile_id='tile_ID', origin =0,
                              destination = 1, flow = 2)
                              
    gravity_singly_fitted.fit(fdf, relevance_column='relevance')
    sc_fdf_fitted = gravity_singly_fitted.generate(tessellation,
                tile_id_column='tile_ID',
                tot_outflows_column='tot_outflow',
                relevance_column= 'relevance',
                out_format='flows')
    l.append(sc_fdf_fitted)

    a+=1
    print(a)

end = time.time()

print("Elapsed time: " + str((end - start)/60) +" minutes")

print(gravity_singly_fitted.deterrence_func_args)

print(gravity_singly_fitted.destination_exp)

/opt/conda/lib/python3.11/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 64/64 [00:00<00:00, 6701.50it/s]
/opt/conda/lib/python3.11/site-packages/skmob/models/gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)
/opt/conda/lib/python3.11/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(


1


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 64/64 [00:00<00:00, 11945.86it/s]
/opt/conda/lib/python3.11/site-packages/skmob/models/gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)
/opt/conda/lib/python3.11/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(


2


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 64/64 [00:00<00:00, 12491.76it/s]
/opt/conda/lib/python3.11/site-packages/skmob/models/gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)
/opt/conda/lib/python3.11/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(


...

/opt/conda/lib/python3.11/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 64/64 [00:00<00:00, 4715.93it/s]
/opt/conda/lib/python3.11/site-packages/skmob/models/gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)
/opt/conda/lib/python3.11/site-packages/statsmodels/genmod/families/links.py:13: FutureWarning: The log link alias is deprecated. Use Log instead. The log link alias will be removed after the 0.15.0 release.
  warnings.warn(


345


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 64/64 [00:00<00:00, 10835.81it/s]

346
Elapsed time: 1.1326209704081218 minutes
[-1.3207649809200892]
0.4517926742324942



/opt/conda/lib/python3.11/site-packages/skmob/models/gravity.py:43: RuntimeWarning: divide by zero encountered in power
  return np.power(x, exponent)


FAKE SET

In [9]:
fake_set = []
for fdf in l:
    adj = toAdj(fdf, tessellation)
    adj = adj.fillna(0)
    arr = np.rint(adj.to_numpy())
    fake_set.append(arr)

with open("./generations/dia/fake_set_gravity.txt", "wb") as fp:   # Unpickling
    pickle.dump(fake_set,fp)

/tmp/ipykernel_495/1352280308.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  adj = adj.fillna(0)
/tmp/ipykernel_495/1352280308.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  adj = adj.fillna(0)
/tmp/ipykernel_495/1352280308.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  adj = adj.fillna(0)
/tmp/ipykernel_495/1352280308.py:4

In [10]:
adj

,0,1,2,3,4,5,6,7,8,9,...,54,55,56,57,58,59,60,61,62,63
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59,0.000505,0.000545,0.000578,0.000601,0.000609,0.0006,0.000578,0.000527,0.000586,0.030063,...,0.329089,0.002747,0.001865,0.16274,0.54435,0.0,0.007952,0.002748,0.005033,0.007953
60,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000
61,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000
62,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00000,0.00000,0.0,0.000000,0.000000,0.000000,0.000000
